In [1]:
import time
import datetime
import pytz
import pandas as pd

from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from numpy import trapz
import numpy as np

from matplotlib import style
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

%matplotlib inline

from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA

import seaborn as sns
style.use("seaborn-notebook")

from xbos import get_client
from xbos.services.pundat import DataClient, timestamp, make_dataframe, merge_dfs
#from xbos.services.hod import HodClientHTTP
from xbos.services.hod import HodClient
from xbos.devices.thermostat import Thermostat
%matplotlib inline

from xbos.services import mdal

## Plot all streams
import matplotlib.dates as mdates
from collections import OrderedDict
from matplotlib.transforms import blended_transform_factory
from __future__ import division
import matplotlib.gridspec as gridspec




In [2]:
##Sites, Weather Stations, Event Dates

SITE_WEATHER_STATION = {"hayward-station-1":"683bf516-1519-359c-92a2-7289d5ce66dc",
             "hayward-station-8":"683bf516-1519-359c-92a2-7289d5ce66dc",
             "ciee":"a73c1b67-142f-3b45-baf8-e308619b6bbc",
             "orinda-community-center":"fd7bbbf6-5f28-321a-99db-def8e84bd4af",
             "word-of-faith-cc":"fa8f400e-766d-30a2-999f-6121eaf0d117",
             "south-berkeley-senior-center":"a73c1b67-142f-3b45-baf8-e308619b6bbc",
             "north-berkeley-senior-center":"a73c1b67-142f-3b45-baf8-e308619b6bbc",
             "berkeley-corporate-yard":"a73c1b67-142f-3b45-baf8-e308619b6bbc",
             "avenal-veterans-hall":"c3d0f042-2cb7-387c-8867-4190fef4fd2c",
             "avenal-public-works-yard":"c3d0f042-2cb7-387c-8867-4190fef4fd2c",
             "avenal-animal-shelter":"c3d0f042-2cb7-387c-8867-4190fef4fd2c",
             "avenal-movie-theatre":"c3d0f042-2cb7-387c-8867-4190fef4fd2c",
             "avenal-recreation-center":"c3d0f042-2cb7-387c-8867-4190fef4fd2c",
             "local-butcher-shop":"a73c1b67-142f-3b45-baf8-e308619b6bbc",
             "rfs":"a73c1b67-142f-3b45-baf8-e308619b6bbc",
             "csu-dominguez-hills":"4335c2f9-fe85-3f63-8310-7d459e448f45",
             "jesse-turner-center":"4eb8a259-0e1f-36ae-af5d-5e62906c905d"}

PGE_SITES = ["hayward-station-1",
             "hayward-station-8",
             "ciee",
             "orinda-community-center",
             "word-of-faith-cc",
             "south-berkeley-senior-center",
             "north-berkeley-senior-center",
             "berkeley-corporate-yard",
             "avenal-veterans-hall",
             "avenal-public-works-yard",
             "avenal-animal-shelter",
             "avenal-movie-theatre",
             "avenal-recreation-center",
             "local-butcher-shop",
             "rfs"]

PGE_PDP = ['06/12/2018', 
              '06/13/2018', 
              '07/10/2018', 
              '07/16/2018', 
              '07/17/2018', 
              '07/19/2018', 
              '07/24/2018', 
              '07/25/2018', 
              '07/27/2018']

SCE_SITES = ["csu-dominguez-hills","jesse-turner-center"]

SCE_CPP = ["07/06/2018",
           "07/09/2018",
           "07/10/2018",
           "07/17/2018",
           "07/18/2018",
           "08/01/2018",
           "08/02/2018",
           "08/06/2018",
           "08/07/2018",
           "08/09/2018",
           "09/28/2018",
           "10/18/2018"]

client = mdal.MDALClient("xbos/mdal")
hod_client = HodClient("xbos/hod")




Saw [xbos/mdal] MDAL 5 seconds 113.827 ms ago
Saw [xbos/hod] HodDB 5 seconds 152.638 ms ago


In [3]:
## query the zones for all of the sites
def get_zones(SITE):
#     c = get_client()


    zones = """
    SELECT ?zone FROM %s WHERE {
    ?zone rdf:type brick:HVAC_Zone .
    };
    """

    res = hod_client.do_query(zones % SITE)
    print res["Rows"]
    return res["Rows"]
#print (res)


In [4]:
def get_data(SITE,ZONE,START,END):

    ###Query for OAT from weather station
    query0 = {
        "Composition": [SITE_WEATHER_STATION[SITE]], # mode and state at MAX
        "Selectors": [mdal.MEAN], #mdal.{MAX, MIN, COUNT, MEAN, RAW} . Define one for each of the variables above
        "Time": {
            "T0": START,
            "T1": END,
            "WindowSize": '5m',
            "Aligned": True,
        },
    }
    try:
        resp = client.do_query(query0,timeout=300)
    except Exception as e:
        return (None,None,None,None,None,None,None)
#     print resp.get('error') # see if there's an error
    df0 = resp['df']
    print df0.describe()

    ###Query for Meter Demand
    query1 = {
       "Composition": ["demand"], # mode and state at MAX
       "Selectors": [mdal.MEAN], #mdal.{MAX, MIN, COUNT, MEAN, RAW} . Define one for each of the variables above
       "Variables": [
           {
            "Name": "demand",
            "Definition": "SELECT ?demand_uuid FROM %s WHERE { ?demand rdf:type brick:Building_Electric_Meter . ?demand bf:uuid ?demand_uuid . };" % SITE,
           },
       ],
       "Time": {
            "T0": START,
            "T1": END,
            "WindowSize": '5m',
            "Aligned": True,
       },
    }
    if SITE != "csu-dominguez-hills":
        try:
            resp = client.do_query(query1,timeout=300)
        except Exception as e:
            return (None,None,None,None,None,None,None)
    #     print resp.get('error') # see if there's an error

    #     resp = client.do_query(query1,timeout=300)
    #     print resp.get('error') # see if there's an error
    #     if resp.get('error')
        df1 = resp['df']
        print df1.describe()

    else:
        df1 = None

    # following query is for the whole building.
    hod_temperature_query = """SELECT ?zone ?uuid FROM %s WHERE { 
          ?tstat rdf:type brick:Thermostat .
          ?tstat bf:controls ?RTU .
          ?RTU rdf:type brick:RTU . 
          ?RTU bf:feeds ?zone. 
          ?zone rdf:type brick:HVAC_Zone . 
          ?tstat bf:hasPoint ?thermostat_point .
          ?thermostat_point rdf:type brick:Temperature_Sensor .
          ?thermostat_point bf:uuid ?uuid.
        };""" % SITE

    temperature_query_data = hod_client.do_query(hod_temperature_query)["Rows"]
    temperature_query_data = {row["?zone"]: row["?uuid"] for row in temperature_query_data} 
    if ZONE not in temperature_query_data:
        print("Zone %s has not UUID",ZONE)
        return (None,None,None,None,None,None,None)
#     temperature_zone_uuid = {row["?zone"]: row["?uuid"] for row in temperature_query_data}[ZONE]
    temperature_zone_uuid = temperature_query_data[ZONE]


    ###Query for Tstat Temperature data IAT
    query2 = {
        "Composition": [temperature_zone_uuid], # zone and temp
        "Selectors": [mdal.MEAN], #mdal.{MAX, MIN, COUNT, MEAN, RAW} . Define one for each of the variables above
        "Time": {
            "T0": START,
            "T1": END,
            "WindowSize": '5m',
            "Aligned": True,
        },
    }
    resp = client.do_query(query2,timeout=300)
    print resp.get('error') # see if there's an error
    df2 = resp['df']
    print df2.describe()

    ###Query for Tstat State data

    hod_state_query = """SELECT ?zone ?uuid FROM %s WHERE { 
      ?tstat rdf:type brick:Thermostat .
      ?tstat bf:controls ?RTU .
      ?RTU rdf:type brick:RTU . 
      ?RTU bf:feeds ?zone. 
      ?zone rdf:type brick:HVAC_Zone . 
      ?tstat bf:hasPoint ?thermostat_point .
      ?thermostat_point rdf:type brick:Thermostat_Status .
      ?thermostat_point bf:uuid ?uuid.
    };""" % SITE

    state_query_data = hod_client.do_query(hod_state_query)["Rows"]
    state_zone_uuid = {row["?zone"]: row["?uuid"] for row in state_query_data}[ZONE]

    query3 = {
        "Composition": [state_zone_uuid], # mode and state at MAX AC-1
        "Selectors": [mdal.MAX], #mdal.{MAX, MIN, COUNT, MEAN, RAW} . Define one for each of the variables above
        "Time": {
            "T0": START,
            "T1": END,
            "WindowSize": '5m',
            "Aligned": True,
        },
    }
    resp = client.do_query(query3,timeout=300)
    print resp.get('error') # see if there's an error
    df3 = resp['df']
    print df3.describe()
    
    ###Query for Tstat Mode data
    hod_mode_query = """SELECT ?zone ?uuid FROM %s WHERE { 
      ?tstat rdf:type brick:Thermostat .
      ?tstat bf:controls ?RTU .
      ?RTU rdf:type brick:RTU . 
      ?RTU bf:feeds ?zone. 
      ?zone rdf:type brick:HVAC_Zone . 
      ?tstat bf:hasPoint ?thermostat_point .
      ?thermostat_point rdf:type brick:Thermostat_Mode_Command .
      ?thermostat_point bf:uuid ?uuid.
    };""" % SITE

    mode_query_data = hod_client.do_query(hod_mode_query)["Rows"]
    mode_zone_uuid = {row["?zone"]: row["?uuid"] for row in mode_query_data}[ZONE]

    query4 = {
        "Composition": [mode_zone_uuid], # mode and state at MAX AC-1
        "Selectors": [mdal.MAX], #mdal.{MAX, MIN, COUNT, MEAN, RAW} . Define one for each of the variables above
        "Time": {
            "T0": START,
            "T1": END,
            "WindowSize": '5m',
            "Aligned": True,
        },
    }
    resp = client.do_query(query4,timeout=300)
    print resp.get('error') # see if there's an error
    df4 = resp['df']
    print df4.describe()


    ###Query for Tstat Cooling Setpoint
        ###Query for Tstat Mode data
    hod_cooling_setpoint_query = """SELECT ?zone ?uuid FROM %s WHERE { 
      ?tstat rdf:type brick:Thermostat .
      ?tstat bf:controls ?RTU .
      ?RTU rdf:type brick:RTU . 
      ?RTU bf:feeds ?zone. 
      ?zone rdf:type brick:HVAC_Zone . 
      ?tstat bf:hasPoint ?thermostat_point .
      ?thermostat_point rdf:type brick:Supply_Air_Temperature_Cooling_Setpoint  .
      ?thermostat_point bf:uuid ?uuid.
    };""" % SITE

    cooling_setpoint_query_data = hod_client.do_query(hod_cooling_setpoint_query)["Rows"]
    cooling_setpoint_zone_uuid = {row["?zone"]: row["?uuid"] for row in cooling_setpoint_query_data}[ZONE]


    query5 = {
        "Composition": [cooling_setpoint_zone_uuid], # mode and state at MAX
        "Selectors": [mdal.MEAN], #mdal.{MAX, MIN, COUNT, MEAN, RAW} . Define one for each of the variables above
        "Time": {
            "T0": START,
            "T1": END,
            "WindowSize": '5m',
            "Aligned": True,
        },
    }
    resp = client.do_query(query5,timeout=300)
    print resp.get('error') # see if there's an error
    df5 = resp['df']
    print df5.describe()


    ###Query for Tstat Heating Setpoint
    hod_heating_setpoint_query = """SELECT ?zone ?uuid FROM %s WHERE { 
      ?tstat rdf:type brick:Thermostat .
      ?tstat bf:controls ?RTU .
      ?RTU rdf:type brick:RTU . 
      ?RTU bf:feeds ?zone. 
      ?zone rdf:type brick:HVAC_Zone . 
      ?tstat bf:hasPoint ?thermostat_point .
      ?thermostat_point rdf:type brick:Supply_Air_Temperature_Heating_Setpoint  .
      ?thermostat_point bf:uuid ?uuid.
    };""" % SITE

    heating_setpoint_query_data = hod_client.do_query(hod_heating_setpoint_query)["Rows"]
    heating_setpoint_zone_uuid = {row["?zone"]: row["?uuid"] for row in heating_setpoint_query_data}[ZONE]



    query6 = {
        "Composition": [heating_setpoint_zone_uuid], # mode and state at MAX
        "Selectors": [mdal.MEAN], #mdal.{MAX, MIN, COUNT, MEAN, RAW} . Define one for each of the variables above
        "Time": {
            "T0": START,
            "T1": END,
            "WindowSize": '5m',
            "Aligned": True,
        },
    }
    resp = client.do_query(query6,timeout=300)
    print resp.get('error') # see if there's an error
    df6 = resp['df']
    print df6.describe()
    return df0,df1,df2,df3,df4,df5,df6



In [5]:
##Plot definitions

def plot_data(df0,df1,df2,df3,df4,df5,df6,SITE,ZONE,Event,start_event,end_event,start,end):
    ## Set time zone 
    matplotlib.rcParams['timezone'] = 'US/Pacific'

    ##filled_markers = ('o', 'v', '^', '<', '>', '8', 's', 'p'(pentagon), '*', 'h'(hexagon), 'H', 'D'(diamond), 'd', 'P', 'X')
    ##lineStyles = {'': '_draw_nothing', ' ': '_draw_nothing', '-': '_draw_solid', '--': '_draw_dashed', '-.': '_draw_dash_dot', ':': '_draw_dotted', 'None': '_draw_nothing'}
    ## for grid ##which : {'major', 'minor', 'both'}; ##axis : {'both', 'x', 'y'}
    #Line2D(xdata, ydata, linewidth=None, linestyle=None, color=None, marker=None, markersize=None, markeredgewidth=None, markeredgecolor=None, markerfacecolor=None, markerfacecoloralt='none', fillstyle=None, antialiased=None, dash_capstyle=None, solid_capstyle=None, dash_joinstyle=None, solid_joinstyle=None, pickradius=5, drawstyle=None, markevery=None, **kwargs)

    fig, ax1 = plt.subplots(figsize=(10, 6), dpi=150, facecolor='w', edgecolor=None, frameon=False) #temperatures IAT & OAT, Heating & Cooling Setpoints
    ax2 = ax1.twinx() #demand in kW
    ax3 = ax1.twinx() #Tstat State
    ax4 = ax1.twinx() #Tstat Mode

    ##OAT = linestyle='-', color='#5ca904' leaf green 
    ###IAT = linestyle='-', color='#b0dd16' yellowish green
    ##demand = linestyle='-', color='#b04e0f' burnt sienna
    ##Mode = linestyle='-', color='m' 
    ##State = linestyle='-', color='#ff9408', tangerine
    ###ClgSP = linestyle='--', color='b',
    ###HtgSP = linestyle='-.', color='r',

    ## OAT Plot Axis 1 (df0)
    lns1 = ax1.plot(df0, linestyle='-', color='#5ca904', linewidth=1.25, markeredgecolor='#5ca904', markerfacecolor=None, marker='+', markeredgewidth=1, markersize=5, label='OAT') #"w+"(white cross). 'k^' (black triangle, markeredgecolor='k', markerfacecolor=None, marker='+', markeredgewidth=1, markersize=5
    ax1.set_ylim(60, 100) # secondary y-axis plot with pandas need to make ylim=(65,115)
    #ymin, ymax = ax1.get_ylim() #if no specifc _ylim set

    ## IAT Plots for each of the TSTATs Axis 1 (df2, df5, df6) also plots for Htg & Clg Setpoints
    lns2 = ax1.plot(df2, linestyle='-', linewidth=1.5, color='#5ca904',
                    marker='s', markevery=5, markeredgecolor='#5ca904', markerfacecolor='w', 
                    markeredgewidth=0.5, markersize=4, label='IAT') #s = square, ^ = triangle, color='#b0dd16'
    lns3 = ax1.plot(df5, linestyle='-', linewidth=1.5, color='k', 
                    marker='^', markevery=10, markeredgecolor='k', markerfacecolor='w', 
                    markeredgewidth=0.5, markersize=5, label='Cooling Setpoint') #dashes=[4,2,4,2,2,2], s = square, ^ = triangle, 
    lns4 = ax1.plot(df6, linestyle='-', linewidth=1.5, color='k',
                    marker='D', markevery=10, markeredgecolor='k', markerfacecolor='w', 
                    markeredgewidth=0.5, markersize=5, label='Heating Setpoint') #dashes=[6,2,2,2,2,2], s = square, ^ = triangle, 

    ## Meter Plot Axis 2 (df1)
    #lns5 = ax2.plot(df1, linestyle='-', color='0.75', linewidth=1, label='Demand') # secondary y-axis plot with pandas need to make ylim=(0,20000)
    ax2.set_ylim(0, 20000)

    ## Tstat State Axis 3 (df3)
    lns6 = ax3.plot(df3, linestyle=':',  color='k', linewidth=1.5, label='State') # secondary y-axis plot with pandas need to make ylim=(0,5)
    ax3.set_ylim(-0.05, 5.05) #dashes=[10, 2, 5, 2]
    ax3.spines['right'].set_position(('axes', -0.2)) # move the 3rd y-axis to the left (0.0):

    ## Tstat Mode Axis 4 (df4) linestyle='--', --mo  markersize=None, markeredgewidth=None, markeredgecolor=None, markerfacecolor=None,
    lns7 = ax4.plot(df4, dashes=[1,1,1,1], color='k', linewidth=3, label='Mode') # secondary y-axis plot with pandas need to make ylim=(0,5)
    ax4.set_ylim(-0.05, 3.05) 
    ax4.spines['right'].set_position(('axes', -0.3)) # move the 3rd y-axis to the left (0.0):

    ## set y-ticks, use np.arange for defined deltas, add a small increment to the last ylim value to ensure that the last value will be a tick
    ax1.set_yticks(np.arange(59, 101, step=5), minor=True)
    ax2.set_yticks(np.arange(0, 20001, step=2500), minor=True)
    ax3.set_yticks(np.arange(0, 5.1, step=1))
    ax4.set_yticks(np.arange(0, 3.1, step=1))

    ## for both left-hand y-axis move the ticks to the outside:
    ax1.get_yaxis().set_tick_params(direction='out')
    ax2.get_yaxis().set_tick_params(direction='out')
    ax3.get_yaxis().set_tick_params(direction='in')
    ax4.get_yaxis().set_tick_params(direction='in')

    ## axis labels, colors (b: blue, g: green, r: red, c: cyan, m: magenta, y: yellow, k: black, w: white)
    ax1.set_xlabel('Time')
    #plt.xticks(rotation=45)
    ax1.tick_params(axis='x', which='major', pad=15, labelrotation=45)

    ax1.set_ylabel('Temp (C)', color='k')
    ax2.set_ylabel('Demand (kW)', color='k')
    ax3.set_ylabel('State', color='k')
    ax4.set_ylabel('Mode', color='k')

    ###Plot locations of DR Events

    #xposition = [pd.to_datetime('2018-09-28 14:00:00 PDT'), pd.to_datetime('2018-09-28 18:00:00 PDT')]
    #for xc in xposition: 
    ax1.axvline(x=pd.to_datetime(start_event), ymin=0, ymax=1, color='g', linestyle=':', lw=1) #ymin=ymin+1, ymax=ymax-1 if no specific limits set
    plt.axvline(x=pd.to_datetime(end_event), ymin=0, ymax=1, color='g', linestyle=':', lw=1) #ymin=ymin+1, ymax=ymax-1 if no specific limits set


    ##Set baseline for the DR event highlights
    start = pd.to_datetime(start) # plot start time
    end = pd.to_datetime(end) # plot end time
    steps = pd.Timedelta(end - start).seconds / 3600.0
    t = np.linspace(start.value, end.value, int(steps)) #total number of hours in the plot
    t = pd.to_datetime(t)

    y1 = ax1.get_ylim()

    mpl_blue_rvb = (191. / 255., 209. / 256., 212. / 255.)
    fill_color = mpl_blue_rvb
    ax1.fill_betweenx(y1, pd.to_datetime(start_event), pd.to_datetime(end_event), color=fill_color, alpha=0.5)

    ###Annotate DR events
    ##Annotation Arrows
    ax1.annotate('DR Event', xy=(start_event, 90), size=10, rotation=0., 
                 bbox=dict(boxstyle="round", ec='w', fc=(191. / 255., 209. / 256., 212. / 255.)),
                 xycoords='data', ha='center', va='center', xytext=(80, 0), textcoords='offset points', 
                 arrowprops=dict(arrowstyle='->', fc='r', ec='r', lw=1)) #, shrink=0.05 only shrink if no arrow style defined
    ##Annotation Arrows
    ax1.annotate('', xy=(end_event, 90), xycoords='data', 
                 ha='center', va='center', xytext=(-60, 0), textcoords='offset points', 
                 arrowprops=dict(arrowstyle='->', fc='r', ec='r', lw=1))  #, shrink=0.05 only shrink if no arrow style defined

    ###Create basic comfort band for OAT and IAT

    ##Plot basic comfort zone lines
    #ax1.axhline(y=68, color='r', linestyle=':', lw=1.5)
    #ax1.axhline(y=80, color='r', linestyle=':', lw=1.5)

    ##Plot basic comfort zone fill
    y2 = 68
    y3 = 80

    plt.rcParams['hatch.color'] = fill_color ##hatch types patterns = ('-', '+', 'x', '\', '\\', '\\\', '//', '*', 'o', 'O', '.', '..', '...')  
    plt.rcParams['hatch.linewidth'] = 0.1 ##hatch line weight
    mpl_blue_rvb = (191. / 255., 209. / 256., 212. / 255.)
    fill_color = mpl_blue_rvb
    ax1.fill_between(t, y2, y3, hatch='...', linewidth=0.0, alpha=0.0) #color=fill_color, alpha=0.5) 
    ## ax1.fill_between(x,y,0.5,where=y>0.5,interpolate=True) # to adjust the fill between a certain range

    ##x axis label
    plt.title(str(SITE)+' | '+str(ZONE)+' | '+str(Event))

    # add these three lines to plot the legend
    lns = lns1+lns2+lns3+lns4+lns6+lns7 #lns5+
    labs = [l.get_label() for l in lns]
    plt.legend(lns, labs, loc='upper center', bbox_to_anchor=(0.5, -0.3), fancybox=False, shadow=False, ncol=3) #bbox_to_anchor=(0.5, -0.1), (1.04,1)

    ## hide spines on top and right
    #ax1.spines['right'].set_visible(False)
    ax1.spines['top'].set_visible(False)
    ax2.spines['top'].set_visible(False)
    ax3.spines['top'].set_visible(False)
    ax4.spines['top'].set_visible(False)

    ##And a corresponding grid, #ax.grid(which='minor', alpha=0.2); #ax.grid(which='major', alpha=0.5)
    ax1.grid(b=True, which='major', alpha=0.5, axis='both', color='0.25', linestyle=':', linewidth=.5)

    ##Locate and Format label of xticks
    #xlocator = mdates.MinuteLocator(interval = 60) #1440 is 1 day

    ax1.xaxis.set_major_locator(mdates.MinuteLocator(interval = 60)) # set major ticks location 
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%m.%d %H:%M')) # set major ticks format

    ax1.xaxis.set_minor_locator(mdates.MinuteLocator(interval = 15)) # set minor ticks location 
    #ax.xaxis.set_minor_formatter(mdates.DateFormatter('%d')) # set minor ticks format

    ##plot graph
    plt.show()

    ##fig = plt.figure() 
    #fig.savefig(str(SITE)+'_'+str(ZONE)+'.png', bbox_inches='tight')
#     fig.savefig("./"+str(SITE)+'_'+str(ZONE)+'_'+str(Event)+'.pdf', bbox_inches='tight')

    fig.savefig(str(SITE)+'_'+str(ZONE)+'_'+str(start_event)+'.pdf', bbox_inches='tight')


In [ ]:
event = SCE_CPP[0] #PGE_PDP[6] #
month, day, year = event.split('/')
start = '{0}-{1}-{2} 09:00:00 PDT'.format(year,month,day)
end = '{0}-{1}-{2} 19:00:00 PDT'.format(year,month,day)
start_event = '{0}-{1}-{2} 14:00:00 PDT'.format(year,month,day)
end_event = '{0}-{1}-{2} 18:00:00 PDT'.format(year,month,day)


site = "csu-dominguez-hills"
zones = get_zones(site)
zone = zones[20]['?zone']
# get_data(site,zone,start,end)
df0,df1,df2,df3,df4,df5,df6 = get_data(site,zone,start,end)
if df0 is not None:
    plot_data(df0,df1,df2,df3,df4,df5,df6,site,zone,event,start_event,end_event,start,end)

##Errors All Sites - 'HVAC_Zone_Outside' (when it gets instantiated in Brick)
## "hayward-station-1": 'HVAC_Zone_UH-3', 'HVAC_Zone_UH-2', 'HVAC_Zone_UH-1' (Zones with no RTUs or TSTATS)
## "hayward-station-8":'HVAC_Zone_UH-2' (Zones with no RTUs or TSTATS)
## "word-of-faith-cc": 'HVAC_Zone_Sanctuary_Rb_2' (turned off? review docs)
## "south-berkeley-senior-center": 'HVAC_Zone_AC_3', HVAC_Zone_AC_2' (Revit comments for RTUs)
## "north-berkeley-senior-center": 'HVAC_Zone_Ac_5', 'HVAC_Zone_Ac_3', '?zone': 'HVAC_Zone_Ac_1' (Revit comments for RTUs)
## "berkeley-corporate-yard":  no errors 
## "avenal-veterans-hall": 'HVAC_Zone_Outside', 'HVAC_Zone_Ac_6', 'HVAC_Zone_Ac_5', 'HVAC_Zone_Ac_4','HVAC_Zone_Ac_3','HVAC_Zone_Ac_2,'HVAC_Zone_Ac_1'
## "avenal-public-works-yard": 'HVAC_Zone_Public_Works_Office', 'HVAC_Zone_Outside'
## "avenal-animal-shelter": 'HVAC_Zone_Outside'
## "avenal-movie-theatre": 'HVAC_Zone_Outside'
## "avenal-recreation-center": 'HVAC_Zone_Outside'
## "local-butcher-shop": no data
## "rfs": 'HVAC_Zone_Michaels_Office' (errors in data IAT is negative)

###
##"csu-dominguez-hills":
##"jesse-turner-center"

In [17]:
try: 
    for event in SCE_CPP:
        month, day, year = event.split('/')
        start = '{0}-{1}-{2} 09:00:00 PDT'.format(year,month,day)
        end = '{0}-{1}-{2} 19:00:00 PDT'.format(year,month,day)
        start_event = '{0}-{1}-{2} 14:00:00 PDT'.format(year,month,day)
        end_event = '{0}-{1}-{2} 18:00:00 PDT'.format(year,month,day)
        for site in SCE_SITES:
            zones = get_zones(site)
            for zone in zones:
                continue
                df0,df1,df2,df3,df4,df5,df6 = get_data(site,zone['?zone'],start,end)
                if df0 is not None:
                    plot_data(df0,df1,df2,df3,df4,df5,df6,site,zone['?zone'],event,start_event,end_event,start,end)
        
    for event in PGE_PDP:
        month, day, year = event.split('/')
        start = '{0}-{1}-{2} 09:00:00 PDT'.format(year,month,day)
        end = '{0}-{1}-{2} 19:00:00 PDT'.format(year,month,day)
        start_event = '{0}-{1}-{2} 14:00:00 PDT'.format(year,month,day)
        end_event = '{0}-{1}-{2} 18:00:00 PDT'.format(year,month,day)
        for site in PGE_SITES:
            zones = get_zones(site)
            for zone in zones:
                    df0,df1,df2,df3,df4,df5,df6 = get_data(site,zone['?zone'],start,end)
                    if df0 is not None:
                        plot_data(df0,df1,df2,df3,df4,df5,df6,site,zone['?zone'],event,start_event,end_event,start,end)
                    
except Exception as e:
    print e
    pass


Query of 
    SELECT ?zone FROM csu-dominguez-hills WHERE {
    ?zone rdf:type brick:HVAC_Zone .
    };
     timed out
